In [70]:
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])} --quiet
sys.path.insert(0, os.path.abspath('./detectron2'))

Cloning into 'detectron2'...
remote: Enumerating objects: 15137, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 15137 (delta 82), reused 109 (delta 64), pack-reused 14979
Receiving objects: 100% (15137/15137), 6.19 MiB | 5.81 MiB/s, done.
Resolving deltas: 100% (10949/10949), done.


In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
cd

/home/ubuntu


In [7]:
cd cw/OneFormer/

/home/ubuntu/cw/OneFormer


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="oneformer")
# Import detectron2 utilities
from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.data import MetadataCatalog


In [12]:
######
#@title 3. Import Libraries and other Utilities
######
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="oneformer")

# Import libraries
import numpy as np
import cv2
import torch
import imutils

# Import detectron2 utilities
from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.data import MetadataCatalog
from OneFormer.demo.defaults import DefaultPredictor
from OneFormer.demo.visualizer import Visualizer, ColorMode


# import OneFormer Project
from oneformer import (
    add_oneformer_config,
    add_common_config,
    add_swin_config,
    add_dinat_config,
    add_convnext_config,
)

In [13]:
device = torch.device("cuda:0")
SWIN_CFG_DICT = {"cityscapes": "configs/cityscapes/oneformer_swin_large_IN21k_384_bs16_90k.yaml",
            "coco": "configs/coco/oneformer_swin_large_IN21k_384_bs16_100ep.yaml",
            "ade20k": "configs/ade20k/oneformer_swin_large_IN21k_384_bs16_160k.yaml",}

DINAT_CFG_DICT = {"cityscapes": "configs/cityscapes/oneformer_dinat_large_bs16_90k.yaml",
            "coco": "configs/coco/oneformer_dinat_large_bs16_100ep.yaml",
            "ade20k": "configs/ade20k/oneformer_dinat_large_IN21k_384_bs16_160k.yaml",}

def setup_cfg(dataset, model_path, use_swin):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_common_config(cfg)
    add_swin_config(cfg)
    add_dinat_config(cfg)
    add_convnext_config(cfg)
    add_oneformer_config(cfg)
    if use_swin:
      cfg_path = SWIN_CFG_DICT[dataset]
    else:
      cfg_path = DINAT_CFG_DICT[dataset]
    cfg.merge_from_file(cfg_path)
    cfg.MODEL.DEVICE = 'cuda:0'
    cfg.MODEL.WEIGHTS = model_path
    cfg.freeze()
    return cfg

def setup_modules(dataset, model_path, use_swin):
    cfg = setup_cfg(dataset, model_path, use_swin)
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get(
        cfg.DATASETS.TEST_PANOPTIC[0] if len(cfg.DATASETS.TEST_PANOPTIC) else "__unused"
    )
    if 'cityscapes_fine_sem_seg_val' in cfg.DATASETS.TEST_PANOPTIC[0]:
        from cityscapesscripts.helpers.labels import labels
        stuff_colors = [k.color for k in labels if k.trainId != 255]
        metadata = metadata.set(stuff_colors=stuff_colors)
    
    return predictor, metadata

def panoptic_run(img, predictor, metadata):
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.IMAGE)
    predictions = predictor(img, "panoptic")
    panoptic_seg, segments_info = predictions["panoptic_seg"]
    out = visualizer.draw_panoptic_seg_predictions(
    panoptic_seg.to(device), segments_info, alpha=0.5
)
    return out

def instance_run(img, predictor, metadata):
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.IMAGE)
    predictions = predictor(img, "instance")
    instances = predictions["instances"].to(device)
    out = visualizer.draw_instance_predictions(predictions=instances, alpha=0.5)
    return out

def semantic_run(img, predictor, metadata):
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.IMAGE)
    predictions = predictor(img, "semantic")
    out = visualizer.draw_sem_seg(
        predictions["sem_seg"].argmax(dim=0).to(device), alpha=0.5
    )
    return out

TASK_INFER = {"panoptic": panoptic_run, 
              "instance": instance_run, 
              "semantic": semantic_run}

In [15]:
use_swin = False
import os
import subprocess
if not use_swin:
  if not os.path.exists("250_16_dinat_l_oneformer_cityscapes_90k.pth"):
    subprocess.run('wget https://shi-labs.com/projects/oneformer/cityscapes/250_16_dinat_l_oneformer_cityscapes_90k.pth', shell=True)
  predictor, metadata = setup_modules("cityscapes", "250_16_dinat_l_oneformer_cityscapes_90k.pth", use_swin)
else:
  if not os.path.exists("250_16_swin_l_oneformer_cityscapes_90k.pth"):
    subprocess.run('wget https://shi-labs.com/projects/oneformer/cityscapes/250_16_swin_l_oneformer_cityscapes_90k.pth', shell=True)
  predictor, metadata = setup_modules("cityscapes", "250_16_swin_l_oneformer_cityscapes_90k.pth", use_swin)

Loading config configs/cityscapes/Base-Cityscapes-UnifiedSegmentation.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[06/30 02:58:55 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from 250_16_dinat_l_oneformer_cityscapes_90k.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  text_encoder.positional_embedding
  text_encoder.transformer.resblocks.0.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.0.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.1.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.1.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.2.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.2.

In [16]:
CITYSCAPES_CLASSES_SETTING = {
    "unlabeled": {"id": 255, "isthing": 0},
    "ego vehicle": {"id": 255, "isthing": 0},
    "rectification border": {"id": 255, "isthing": 0},
    "out of roi": {"id": 255, "isthing": 0},
    "static": {"id": 255, "isthing": 0},
    "dynamic": {"id": 255, "isthing": 0},
    "ground": {"id": 255, "isthing": 0},
    "road": {"id": 0, "isthing": 0},
    "sidewalk": {"id": 1, "isthing": 0},
    "parking": {"id": 255, "isthing": 0},
    "rail track": {"id": 255, "isthing": 0},
    "building": {"id": 2, "isthing": 0},
    "wall": {"id": 3, "isthing": 0},
    "fence": {"id": 4, "isthing": 0},
    "guard rail": {"id": 255, "isthing": 0},
    "bridge":  {"id": 255, "isthing": 0},
    "tunnel": {"id": 255, "isthing": 0},
    "pole": {"id": 5, "isthing": 0},
    "polegroup": {"id": 255, "isthing": 0},
    "traffic light": {"id": 6, "isthing": 0},
    "traffic sign": {"id": 7, "isthing": 0},
    "vegetation": {"id": 8, "isthing": 0},
    "terrain": {"id": 9, "isthing": 0},
    "sky": {"id": 10, "isthing": 0},
    "person": {"id": 11, "isthing": 1},
    "rider": {"id": 12, "isthing": 1},
    "car": {"id": 13, "isthing": 1},
    "truck": {"id": 14, "isthing": 1},
    "bus": {"id": 15, "isthing": 1},
    "caravan": {"id": 255, "isthing": 1},
    "trailer": {"id": 255, "isthing": 0},
    "train": {"id": 16, "isthing": 0},
    "motorcycle": {"id": 17, "isthing": 1},
    "bicycle": {"id": 18, "isthing": 1},
    "license plate": {"id": -1, "isthing": 0}
}

In [21]:
from pycocotools import mask as coco_mask

import json
import numpy as np

input_path = "/home/ubuntu/cw/OneFormer/samples/230622_5"
output_path = "./oneformer_output/"
os.makedirs(output_path, exist_ok=True)

img_files = os.listdir(input_path)
img_files.sort()

annotations = []
categories = []
images = []
image_id = 1

for i, filename in enumerate(os.listdir(input_path)):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        input_filepath = os.path.join(input_path, filename)
        img = cv2.imread(input_filepath)
        img = imutils.resize(img, width=512)
        task = "panoptic"
        device = torch.device('cuda:0')
        predictions = predictor(img, task)
        
        annotation = {}
        instances = predictions["instances"]
        masks = instances.pred_masks
        boxes = instances.pred_boxes.tensor
        scores = instances.scores
        classes = instances.pred_classes
        size_list = []
        num_instances = len(predictions["panoptic_seg"][1])
        
        sem_seg = predictions["sem_seg"].argmax(dim=0).cpu().numpy()
        for sem_seg_val in np.unique(sem_seg):
            mask = (sem_seg == sem_seg_val).astype(np.uint8)
            binary_mask = np.expand_dims(mask, axis=-1)
            rle = coco_mask.encode(np.asfortranarray(binary_mask))
            area = float(np.sum(binary_mask))
            bbox = cv2.boundingRect(mask)
            x, y, w, h = bbox
            bbox_list = list([x,y,w,h])
            counts_decoded = rle[0]['counts'].decode('utf-8')
            counts = ''.join(counts_decoded.split())
            category_id = None
            for i, seg_info in enumerate(predictions['panoptic_seg'][1]):
                if seg_info['category_id'] == sem_seg_val:
                    category_id = seg_info['category_id']                    
                    break
            if category_id is None:
                continue
            segment_field = {
                'counts': counts,
                'size': [rle[0]['size'][0], rle[0]['size'][1]]
            }
            annotation = {
                "area": area,
                "category_id": int(category_id),
                "id": int(sem_seg_val),
                "bbox": bbox_list,
                "image_id": image_id,
                "is_crowd": 0,
                "segmentation": segment_field
            }
            annotations.append(annotation)

        for j in range(num_instances):
            categoryid = predictions["panoptic_seg"][1][j]
            category_id = int(categoryid["category_id"])
            if len(masks) <= j:
                continue
            mask = np.asarray(masks[j].cpu().numpy(), dtype=np.uint8)
            binary_mask = np.zeros((mask.shape[0], mask.shape[1], 1), dtype=np.uint8)
            binary_mask[mask >0.5] = 1
            rle = coco_mask.encode(np.asfortranarray(binary_mask))
            area = float(np.sum(binary_mask))
            bbox = boxes[j].tolist()
            bbox[2] -= bbox[0]
            bbox[3] -= bbox[1]
            annotation = {
                "category_id": int(category_id),
                "id": j + len(np.unique(sem_seg)),
                "bbox": bbox,
                "image_id": image_id,
                "is_crowd": 0
            }
            if instances.has("pred_boxes") and len(instances.pred_boxes) > j:
                counts_decoded = rle[0]['counts'].decode('utf-8')
                counts = ''.join(counts_decoded.split())
                segment_field = {
                    'counts': counts,
                    'size': [rle[0]['size'][0], rle[0]['size'][1]]
                }
                annotation["segmentation"] = segment_field
            if len(scores) > j:
                annotation["score"] = float(scores[j])
            else:
                annotation["score"] = 0
            if len(classes) > j:
                annotation["category_id"] = int(classes[j])
            annotations.append(annotation)
        
        image = {}
        image["id"] = image_id
        image["width"] = int(predictions["instances"].image_size[1])
        image["height"] = int(predictions["instances"].image_size[0])
        image["file_name"] = filename
        images.append(image)
        image_id += 1

for name, setting in CITYSCAPES_CLASSES_SETTING.items():
    categories.append({
        'id': setting['id'],
        'name': name,
        'isthing': setting['isthing']
    })

coco_dict = {}
coco_dict["annotations"] = annotations
coco_dict["categories"] = categories
coco_dict["images"] = images

# output_filename = "oneformer_coco_230425.json"
# output_filename = "oneformer_raw_data_230427_3.json"
# output_filename = "oneformer_0508_TEST.json"
output_filename = "oneformer_230622_5.json"
output_filepath = os.path.join(output_path, output_filename)

with open(output_filepath, "w", encoding="utf-8") as f:
    json.dump(coco_dict, f)

print(f"Output coco json saved at {output_filepath}")

Output coco json saved at ./oneformer_output/oneformer_230622_5.json
